<a href="https://colab.research.google.com/github/Gajendra-theDataEngineer004627/All_Learnings/blob/main/Final_Project_WebScrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [26]:
#---------------------------------Extracting Name of the Product---------------------------#
# Function to extract Product Title
def get_title(soup):
    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        # Inner NavigatableString Object
        title_value = title.text
        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""
    return title_string

#--------------------------------Extracting Product Price----------------------------------#
# Function to extract Product Price
def get_price(soup):
    try:
        price = soup.find("span", attrs={"class":'a-price aok-align-center reinventPricePriceToPayMargin priceToPay'}).find("span", attrs={"class": "a-offscreen"}).string.strip()
    except AttributeError:
        try:
            # If there is some deal price
            price = soup.find("span", attrs={"class":'a-price aok-align-center reinventPricePriceToPayMargin priceToPay'}).find("span", attrs={"class": "a-offscreen"}).string.strip()
        except:
            price = ""
    return price

#-------------------------------Extracting Ratings -----------------------#
# Function to extract Product Rating
def get_rating(soup):
    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""
    return rating


#------------------------------------Extracting Number of users reviewed -------------#
# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()
    except AttributeError:
        review_count = ""

    return review_count

#----------------------------------Extract Availability Status ----------------#

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"

    return available

In [28]:
if __name__ == '__main__':

    # add your user agent
    # add your own user aget for better analysis below wont work for you
    HEADERS = ({'User-Agent':'Mozila/5.0 (Windows NT 10.0; Win54; x64) AppleWebKit/514.33 (KHTML, like Gecko) Chrome/119.0.5.0 Safari/524.36', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    url = "https://www.amazon.in/s?k=mobiles&ref=nb_sb_noss"

    # HTTP Request
    webpage = requests.get(url, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}

    # Loop for extracting product details from each link
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.in" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))



# Assuming you already have a dictionary 'd'
        amazon_df = pd.DataFrame.from_dict(d)

# Replace empty titles with NaN in the 'title' column
        amazon_df['title'].replace('', np.nan, inplace=True)

# Drop rows with NaN values in the 'title' column
        amazon_df.dropna(subset=['title'], inplace=True)

# Sort the DataFrame by 'reviews' column in descending order
        final_amazon_df = amazon_df.sort_values(by='reviews', ascending=False)

# Export the sorted DataFrame to a CSV file
        final_amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [29]:
final_amazon_df

,title,price,rating,reviews,availability
21,"Samsung Galaxy A14 5G (Dark Red, 8GB, 128GB St...","₹19,999",3.9 out of 5 stars,944 ratings,In stock
8,"Samsung Galaxy A14 5G (Black, 8GB, 128GB Stora...","₹19,999",3.9 out of 5 stars,944 ratings,In stock
6,"Samsung Galaxy M14 5G (Smoky Teal, 6GB, 128GB ...","₹12,990",4.0 out of 5 stars,"8,734 ratings",In stock
19,"realme narzo N55 (Prime Black, 6GB+128GB) 33W ...","₹12,999",4.0 out of 5 stars,"8,117 ratings",In stock
20,Redmi Note 12 5G Frosted Green 4GB RAM 128GB R...,"₹15,998",3.8 out of 5 stars,"7,836 ratings",In stock
1,"Redmi 12C (Mint Green, 4GB RAM, 64GB Storage) ...","₹8,299",3.9 out of 5 stars,"7,826 ratings",In stock
17,"realme narzo N53 (Feather Gold, 4GB+64GB) 33W ...","₹8,999",3.9 out of 5 stars,"7,423 ratings",In stock
9,"realme narzo N53 (Feather Black, 4GB+64GB) 33W...","₹8,999",3.9 out of 5 stars,"7,423 ratings",In stock
16,"Lava Blaze 5G (Glass Green, 4GB RAM, UFS 2.2 1...","₹9,999",4.0 out of 5 stars,"5,121 ratings",In stock
12,"Oppo A17k (Blue, 3GB RAM, 64GB Storage) with N...","₹8,999",4.0 out of 5 stars,380 ratings,In stock


import numpy as np
from google.colab import autoviz

def categorical_histogram(df, colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  df.groupby(colname).size().plot(kind='barh', color=sns.palettes.mpl_palette(mpl_palette_name), figsize=(8*figscale, 4.8*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  return autoviz.MplChart.from_current_mpl_state()

chart = categorical_histogram(final_amazon_df, *['rating'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_12, *['index'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(_df_13, *['index'], **{})
chart

import numpy as np
from google.colab import autoviz

def categorical_histogram(df, colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  df.groupby(colname).size().plot(kind='barh', color=sns.palettes.mpl_palette(mpl_palette_name), figsize=(8*figscale, 4.8*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  return autoviz.MplChart.from_current_mpl_state()

chart = categorical_histogram(_df_14, *['rating'], **{})
chart

import numpy as np
from google.colab import autoviz

def violin_plot(df, value_colname, facet_colname, figscale=1, mpl_palette_name='Dark2', **kwargs):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (12 * figscale, 1.2 * figscale * len(df[facet_colname].unique()))
  plt.figure(figsize=figsize)
  sns.violinplot(df, x=value_colname, y=facet_colname, palette=mpl_palette_name, **kwargs)
  sns.despine(top=True, right=True, bottom=True, left=True)
  return autoviz.MplChart.from_current_mpl_state()

chart = violin_plot(_df_15, *['index', 'rating'], **{'inner': 'stick'})
chart